In [1]:
import pandas as pd

In [49]:
#geo file
geo = pd.read_csv("/data/discover/Data/General/general_measures.csv", 
                     dtype={'year':str,'state_fips': object,'county_fips':object,
                            'tract_fips':object,'GEO_ID':str,'area_type':str,'Geo_name':str}, low_memory=False)

#keep geo variables
geo=geo[['state_fips','GEO_ID','area_type','Geo_name']]

#drop duplicates
geo = geo.drop_duplicates()

In [52]:
# Job creation

#United States
us_bgr = pd.read_csv("/data/explore/nwank002/Business_Indicators/Business_Growth_Rate/us_bgr.csv", 
                     dtype={'year':str}, low_memory=False)

us_bgr["area_type"] = "national"
us_bgr["GEO_ID"] = "us"
us_bgr["Geo_name"] = "United States"

#State
state_bgr = pd.read_csv("/data/explore/nwank002/Business_Indicators/Business_Growth_Rate/state_bgr.csv", 
                     dtype={'year':str}, low_memory=False)

state_bgr.rename(columns = {"name":"Geo_name"}, inplace = True)

#Merge to geo_file
state_bgr_merge = pd.merge(state_bgr, geo, on=['Geo_name'],
                     how='left')

job_growth = pd.concat([us_bgr,state_bgr_merge], ignore_index=True)

# Make all column names uppercase
job_growth.columns=map(str.upper, job_growth.columns)


In [53]:
#Firm Growth

#United States
us_firm_growth = pd.read_csv("/data/explore/nwank002/Business_Indicators/Business_Growth_Rate/us_firm_growth.csv", 
                     dtype={'year':str}, low_memory=False)

us_firm_growth["area_type"] = "national"
us_firm_growth["GEO_ID"] = "us"
us_firm_growth["Geo_name"] = "United States"

#State
state_firm_growth = pd.read_csv("/data/explore/nwank002/Business_Indicators/Business_Growth_Rate/state_firm_growth.csv", 
                     dtype={'year':str}, low_memory=False)

state_firm_growth.rename(columns = {"name":"Geo_name"}, inplace = True)

#Merge to geo_file
state_firm_growth_merge = pd.merge(state_firm_growth, geo, on=['Geo_name'],
                     how='left')

firm_growth = pd.concat([us_firm_growth,state_firm_growth_merge], ignore_index=True)

# Make all column names uppercase
firm_growth.columns=map(str.upper, firm_growth.columns)

In [54]:
#Job rate

#United States
us_job_rate = pd.read_csv("/data/explore/nwank002/Business_Indicators/Business_Growth_Rate/us_job_rate.csv", 
                     dtype={'year':str}, low_memory=False)

us_job_rate = us_job_rate[us_job_rate.name.str.contains("United States")]

us_job_rate["area_type"] = "national"
us_job_rate["GEO_ID"] = "us"
us_job_rate.rename(columns = {"name":"Geo_name"}, inplace = True)

#State
state_job_rate = pd.read_csv("/data/explore/nwank002/Business_Indicators/Business_Growth_Rate/state_job_rate.csv", 
                     dtype={'year':str}, low_memory=False)

state_job_rate.rename(columns = {"name":"Geo_name"}, inplace = True)

#Merge to geo_file
state_job_rate_merge = pd.merge(state_job_rate, geo, on=['Geo_name'],
                     how='left')

job_rate = pd.concat([us_job_rate,state_job_rate_merge], ignore_index=True)

# Make all column names uppercase
job_rate.columns=map(str.upper, job_rate.columns)

In [63]:
#Imports and Exports

#United States
import_export = pd.read_excel("/data/explore/dataExtracts/data/international_balance_exports_imports.xlsx", sheet_name='CIDR', 
                     dtype={'Year':str})

import_export=import_export[['Year','Balance','Exports','Imports']]

import_export["area_type"] = "national"
import_export["GEO_ID"] = "us"
import_export["Geo_name"] = "United States"

# Make all column names uppercase
import_export.columns=map(str.upper, import_export.columns)

In [90]:
#GDP
GDP = pd.read_csv("/data/explore/nwank002/Business_Indicators/GDP/GrossDomesticProduct.csv", 
                     low_memory=False)

GDP=GDP[['GeoName','2015','2016','2017','2018','2019','2020','2021','2022']]

GDP_melt = GDP.melt(id_vars=['GeoName'])
GDP_melt.rename(columns = {"GeoName":"Geo_name", "variable":"year", "value":"GDP"}, inplace = True)

#Merge to geo_file
GDP_merge = pd.merge(GDP_melt, geo, on=['Geo_name'],
                     how='left')

# Make all column names uppercase
GDP_merge.columns=map(str.upper, GDP_merge.columns)

#Make GDP integer
GDP_merge["GDP"] = (GDP_merge["GDP"].str.replace(",", "")).astype(int)

In [91]:
#Merge
economic = pd.merge(job_growth, firm_growth, on=['GEO_ID','YEAR','AREA_TYPE','GEO_NAME','STATE_FIPS'],
                     how='outer').merge(job_rate, on=['GEO_ID','YEAR','AREA_TYPE','GEO_NAME','STATE_FIPS'],
                     how='outer').merge(import_export, on=['GEO_ID','YEAR','AREA_TYPE','GEO_NAME'],
                     how='outer').merge(GDP_merge, on=['GEO_ID','YEAR','AREA_TYPE','GEO_NAME','STATE_FIPS'],
                     how='outer')

In [92]:
#Save in folder
economic.to_csv("/data/discover/Data/Final Data/Economic Vitality/economic.csv", 
                         header=True, index=False)